In [11]:
from Bio import SeqIO
import pandas as pd
import os

In [12]:
contigs = SeqIO.to_dict(SeqIO.parse("/home/tobamo/analize/project-tobamo/analysis/data/contigs/contigs_non_cellular_filtered.fasta", "fasta"))
gt = pd.read_excel("/home/tobamo/analize/project-tobamo/analysis/data/domain_sci_input/ground_truth_final_added_categories.xlsx")

novel_tobamo = gt[gt["category"] == "tob2"]["contig_name"].tolist()
tob2_contigs = [contig for k,contig in contigs.items() if contig.id in novel_tobamo]

os.makedirs("/home/tobamo/analize/project-tobamo/analysis/pplacer/data/", exist_ok=True)
SeqIO.write(tob2_contigs, "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_contigs.fasta", "fasta")

105

# Pplacer Implementation for Tobamovirus Analysis

This notebook implements phylogenetic placement using pplacer to analyze novel tobamovirus contigs (`tob2_contigs`) against a reference phylogeny.

## Overview
Pplacer places query sequences on a fixed reference phylogenetic tree according to a reference alignment. It finds the optimal attachment location and pendant branch length that maximize the likelihood.

## Workflow Steps:
1. Install pplacer and dependencies
2. Prepare reference sequences and alignment
3. Build reference tree
4. Create reference package
5. Run pplacer on tob2_contigs
6. Analyze results with guppy

In [13]:
# Check pplacer installation and install if needed
import subprocess
import sys
import os
from pathlib import Path

def check_command(cmd):
    """Check if a command is available"""
    try:
        subprocess.run(cmd, capture_output=True, check=True)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

# Check if pplacer and guppy are available
pplacer_available = check_command(['pplacer', '--version'])
guppy_available = check_command(['guppy', '--version'])

print(f"pplacer available: {pplacer_available}")
print(f"guppy available: {guppy_available}")

if not pplacer_available or not guppy_available:
    print("\nPlacer suite not found. Installing via conda...")
    # Check if conda is available
    conda_available = check_command(['conda', '--version'])
    if conda_available:
        print("Installing pplacer via conda...")
        subprocess.run(['conda', 'install', '-c', 'bioconda', 'pplacer', '-y'], check=True)
    else:
        print("Conda not available. Please install pplacer manually:")
        print("1. Download from: https://github.com/matsen/pplacer/releases/latest")
        print("2. Or use conda: conda install -c bioconda pplacer")
        print("3. Or build from source following: https://matsen.github.io/pplacer/compiling.html")

pplacer available: False
guppy available: False

Placer suite not found. Installing via conda...
Installing pplacer via conda...
Retrieving notices: ...working... done
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.9.1





## Package Plan ##

  environment location: /home/tobamo/miniconda3/envs/tobamo-model

  added / updated specs:
    - pplacer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    openssl-3.5.4              |       h26f9b46_0         3.0 MB  conda-forge
    pplacer-1.1.alpha19        |       h9ee0642_2         8.6 MB  bioconda
    ------------------------------------------------------------
                                           Total:        11.7 MB

The following NEW packages will be INSTALLED:

  pplacer            bioconda/linux-64::pplacer-1.1.alpha19-h9ee0642_2 

The following packages will be UPDATED:

  ca-certificates                       2025.8.3-hbd8a1cb_0 --> 2025.10.5-hbd8a1cb_0 
  openssl                                  3.5.3-h26f9b46_1 --> 3.5.4-h26f9b46_0 



ca-certificates-2025 | 152 KB    

In [14]:
# Check if we need additional software for alignment and tree building
# We'll need MAFFT for alignment and FastTree for phylogeny

def install_if_missing(cmd, package_name, conda_package=None):
    """Install package if command not found"""
    if not check_command([cmd, '--version']):
        print(f"{cmd} not found. Installing {package_name}...")
        if conda_package and check_command(['conda', '--version']):
            subprocess.run(['conda', 'install', '-c', 'bioconda', conda_package, '-y'], check=True)
            return True
        else:
            print(f"Please install {package_name} manually")
            return False
    else:
        print(f"{cmd} is available")
        return True

# Check and install required tools
mafft_ok = install_if_missing('mafft', 'MAFFT', 'mafft')
fasttree_ok = install_if_missing('FastTree', 'FastTree', 'fasttree')

if mafft_ok and fasttree_ok:
    print("All required tools are available!")
else:
    print("Some tools are missing. Please install them manually.")

mafft is available
FastTree not found. Installing FastTree...
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.9.1





## Package Plan ##

  environment location: /home/tobamo/miniconda3/envs/tobamo-model

  added / updated specs:
    - fasttree


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fasttree-2.2.0             |       h7b50bb2_0         200 KB  bioconda
    ------------------------------------------------------------
                                           Total:         200 KB

The following NEW packages will be INSTALLED:

  fasttree           bioconda/linux-64::fasttree-2.2.0-h7b50bb2_0 



                                                     
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
All required tools are available!


In [15]:
# Create multiple sequence alignment of reference sequences
import tempfile

print("Creating multiple sequence alignment of reference sequences...")

# Paths
ref_fasta = "/home/tobamo/analize/project-tobamo/analysis/data/tobamo/reference_nukleotidne.fasta"
ref_alignment = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta"

# Run MAFFT alignment
mafft_cmd = [
    'mafft', 
    '--auto',           # Automatically select strategy
    '--adjustdirection', # Reverse complement if needed
    '--thread', '4',    # Use 4 threads
    ref_fasta
]

print(f"Running MAFFT: {' '.join(mafft_cmd)}")
try:
    with open(ref_alignment, 'w') as outfile:
        result = subprocess.run(mafft_cmd, stdout=outfile, stderr=subprocess.PIPE, text=True, check=True)
    print(f"Alignment completed successfully!")
    print(f"Alignment saved to: {ref_alignment}")
    
    # Check alignment
    aligned_seqs = list(SeqIO.parse(ref_alignment, "fasta"))
    if aligned_seqs:
        print(f"Aligned {len(aligned_seqs)} sequences")
        print(f"Alignment length: {len(aligned_seqs[0].seq)} positions")
    else:
        print("Warning: No sequences in alignment file")
        
except subprocess.CalledProcessError as e:
    print(f"MAFFT failed with error: {e}")
    print(f"stderr: {e.stderr}")

Creating multiple sequence alignment of reference sequences...
Running MAFFT: mafft --auto --adjustdirection --thread 4 /home/tobamo/analize/project-tobamo/analysis/data/tobamo/reference_nukleotidne.fasta
Alignment completed successfully!
Alignment saved to: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta
Aligned 125 sequences
Alignment length: 14162 positions


In [40]:
# Build reference phylogenetic tree - use working IQ-TREE, then try pplacer without stats
print("Building reference phylogenetic tree...")

ref_alignment = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta"
ref_tree = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre"
iqtree_prefix = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree"

print("FastTree has compatibility issues on this system")
print("Using IQ-TREE (will run pplacer without stats file)")

# IQ-TREE command with --redo to overwrite existing files
iqtree_cmd = [
    'iqtree',
    '-s', ref_alignment,     # Input alignment
    '-m', 'GTR+G',          # GTR model 
    '--prefix', iqtree_prefix,  # Output prefix
    '--redo',               # Overwrite existing analysis
    '-T', '4',              # Use 4 threads
    '-quiet'                # Reduce output verbosity
]

print(f"Running IQ-TREE: {' '.join(iqtree_cmd)}")
try:
    result = subprocess.run(iqtree_cmd, capture_output=True, text=True, check=True)
    
    # IQ-TREE creates .treefile, rename to .tre for consistency
    iqtree_output = f"{iqtree_prefix}.treefile"
    if os.path.exists(iqtree_output):
        import shutil
        shutil.move(iqtree_output, ref_tree)
        print(f"✓ Tree file renamed from {iqtree_output} to {ref_tree}")
    
    # Check tree file
    if os.path.exists(ref_tree):
        with open(ref_tree, 'r') as f:
            tree_content = f.read().strip()
            if tree_content:
                print(f"✓ Tree building completed successfully!")
                print(f"✓ Tree saved to: {ref_tree}")
                print(f"Tree length: {len(tree_content)} characters")
                print(f"Tree preview: {tree_content[:100]}...")
            else:
                print("⚠ Warning: Tree file is empty")
    else:
        print("⚠ Warning: Tree file not created")
        
except subprocess.CalledProcessError as e:
    print(f"❌ IQ-TREE failed with error: {e}")
    print(f"stdout: {e.stdout}")
    print(f"stderr: {e.stderr}")

# Final status
if os.path.exists(ref_tree) and os.path.getsize(ref_tree) > 0:
    tree_size = os.path.getsize(ref_tree)
    print(f"\n✅ Tree ready: {ref_tree} ({tree_size:,} bytes)")
    print("Strategy: Run pplacer without stats file (it will estimate GTR parameters)")
else:
    print("\n❌ No tree available for pplacer")

print("\nNote: pplacer should work without stats files by estimating parameters")

Building reference phylogenetic tree...
FastTree has compatibility issues on this system
Using IQ-TREE (will run pplacer without stats file)
Running IQ-TREE: iqtree -s /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta -m GTR+G --prefix /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree --redo -T 4 -quiet
✓ Tree file renamed from /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.treefile to /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre
✓ Tree building completed successfully!
✓ Tree saved to: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre
Tree length: 13965 characters
Tree preview: (X03854.1_Barley_stripe_mosaic_virus__BSMV__Type_strain__genomic_RNA_beta:0.7906205365,(((((((((((X1...

✅ Tree ready: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre (13,966 bytes)
Strategy: Run pplacer without stats file (it will estimate GTR

In [18]:
# Align query sequences (tob2_contigs) to the reference alignment
print("Aligning query sequences to reference...")

ref_alignment = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta"
query_fasta = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_contigs.fasta"
combined_alignment = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/combined_alignment.fasta"

# Option 1: Use MAFFT --add to add query sequences to existing alignment
mafft_add_cmd = [
    'mafft',
    '--add', query_fasta,    # Add these sequences
    '--reorder',             # Reorder sequences
    '--thread', '4',
    ref_alignment            # To this reference alignment
]

print(f"Running MAFFT --add: {' '.join(mafft_add_cmd)}")
try:
    with open(combined_alignment, 'w') as outfile:
        result = subprocess.run(mafft_add_cmd, stdout=outfile, stderr=subprocess.PIPE, text=True, check=True)
    
    print(f"Combined alignment completed successfully!")
    print(f"Combined alignment saved to: {combined_alignment}")
    
    # Check combined alignment
    combined_seqs = list(SeqIO.parse(combined_alignment, "fasta"))
    ref_seqs = list(SeqIO.parse(ref_alignment, "fasta"))
    query_seqs = list(SeqIO.parse(query_fasta, "fasta"))
    
    print(f"Reference sequences: {len(ref_seqs)}")
    print(f"Query sequences: {len(query_seqs)}")
    print(f"Combined sequences: {len(combined_seqs)}")
    print(f"Combined alignment length: {len(combined_seqs[0].seq)} positions")
    
    # Verify we have all sequences
    expected_total = len(ref_seqs) + len(query_seqs)
    if len(combined_seqs) == expected_total:
        print("✓ All sequences successfully included in combined alignment")
    else:
        print(f"⚠ Warning: Expected {expected_total} sequences, got {len(combined_seqs)}")
        
except subprocess.CalledProcessError as e:
    print(f"MAFFT --add failed with error: {e}")
    print(f"stderr: {e.stderr}")

Aligning query sequences to reference...
Running MAFFT --add: mafft --add /home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_contigs.fasta --reorder --thread 4 /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta
Combined alignment completed successfully!
Combined alignment saved to: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/combined_alignment.fasta
Reference sequences: 125
Query sequences: 105
Combined sequences: 230
Combined alignment length: 16716 positions
✓ All sequences successfully included in combined alignment


In [29]:
# Create reference package using rppr
print("Creating reference package...")

# Paths
ref_tree = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre"
ref_log = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.log"
ref_alignment = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta"
refpkg_dir = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tobamo.refpkg"

# Remove existing refpkg if it exists
import shutil
if os.path.exists(refpkg_dir):
    shutil.rmtree(refpkg_dir)

# Create reference package with rppr
rppr_cmd = [
    'rppr', 'create',
    '-c', refpkg_dir,           # Output reference package directory
    '-t', ref_tree,             # Reference tree
    '-r', ref_alignment,        # Reference alignment
    '-s', ref_log               # Statistics file from FastTree
]

print(f"Running rppr create: {' '.join(rppr_cmd)}")
try:
    result = subprocess.run(rppr_cmd, capture_output=True, text=True, check=True)
    
    print("Reference package created successfully!")
    print(f"Reference package: {refpkg_dir}")
    
    # Check what's in the reference package
    if os.path.exists(refpkg_dir):
        refpkg_contents = os.listdir(refpkg_dir)
        print(f"Reference package contents: {refpkg_contents}")
    
    # Print rppr output if any
    if result.stdout:
        print(f"rppr stdout: {result.stdout}")
        
except subprocess.CalledProcessError as e:
    print(f"rppr create failed with error: {e}")
    print(f"stdout: {e.stdout}")
    print(f"stderr: {e.stderr}")
    
    # Try alternative approach without statistics file
    print("\nTrying without statistics file...")
    rppr_simple_cmd = [
        'rppr', 'create',
        '-c', refpkg_dir,
        '-t', ref_tree,
        '-r', ref_alignment
    ]
    
    try:
        result = subprocess.run(rppr_simple_cmd, capture_output=True, text=True, check=True)
        print("Reference package created successfully (without stats)!")
    except subprocess.CalledProcessError as e2:
        print(f"rppr create also failed without stats: {e2}")
        print(f"stderr: {e2.stderr}")

Creating reference package...
Running rppr create: rppr create -c /home/tobamo/analize/project-tobamo/analysis/pplacer/data/tobamo.refpkg -t /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre -r /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta -s /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.log
rppr create failed with error: Command '['rppr', 'create', '-c', '/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tobamo.refpkg', '-t', '/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre', '-r', '/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta', '-s', '/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.log']' returned non-zero exit status 1.
stdout: Unknown rppr command: create
Here is a list of commands available using this interface:
  rppr
    check          checks a reference package
    convex_taxids  determin

In [41]:
# Step 1: Setup paths and check existing files
print("Setting up pplacer analysis...")

ref_alignment = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta"
ref_tree = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre"
ref_log = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.log"  # FastTree log file
query_fasta = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_contigs.fasta"
query_only_aligned = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_queries_only.fasta"
placements_file = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_placements.jplace"

# Check which files exist
files_to_check = {
    "Reference alignment": ref_alignment,
    "Reference tree": ref_tree,
    "FastTree log": ref_log,          # Changed from IQ-TREE stats
    "Query sequences": query_fasta,
    "Query alignment": query_only_aligned
}

print("Checking file availability:")
for name, path in files_to_check.items():
    exists = os.path.exists(path)
    status = "✓" if exists else "✗"
    print(f"  {status} {name}: {path}")
    
    if exists and name != "Query alignment":  # Show size for existing files except query alignment
        size = os.path.getsize(path)
        print(f"    Size: {size:,} bytes")

print(f"\nTarget placement file: {placements_file}")
print("Note: Now using FastTree log file for pplacer compatibility")

Setting up pplacer analysis...
Checking file availability:
  ✓ Reference alignment: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta
    Size: 1,810,508 bytes
  ✓ Reference tree: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre
    Size: 13,966 bytes
  ✓ FastTree log: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.log
    Size: 64,680 bytes
  ✓ Query sequences: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_contigs.fasta
    Size: 141,244 bytes
  ✓ Query alignment: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_queries_only.fasta

Target placement file: /home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_placements.jplace
Note: Now using FastTree log file for pplacer compatibility


In [34]:
# Step 2: Prepare query alignment (only if needed)
print("Preparing query sequences for pplacer...")

if os.path.exists(query_only_aligned):
    print("✓ Using existing query-only alignment...")
    query_records = list(SeqIO.parse(query_only_aligned, "fasta"))
    print(f"  Found {len(query_records)} aligned query sequences")
    print(f"  Alignment length: {len(query_records[0].seq)} positions")
else:
    print("Creating new query-only alignment...")
    
    # Get query sequence IDs
    query_ids = set()
    for record in SeqIO.parse(query_fasta, "fasta"):
        query_ids.add(record.id)
    
    print(f"  Found {len(query_ids)} query sequences to align")
    
    # Align queries to reference using MAFFT --addfragments
    temp_aligned = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/temp_aligned.fasta"
    mafft_align_cmd = [
        'mafft',
        '--addfragments', query_fasta,
        '--thread', '4',
        ref_alignment
    ]
    
    print(f"  Running MAFFT: {' '.join(mafft_align_cmd)}")
    with open(temp_aligned, 'w') as outfile:
        subprocess.run(mafft_align_cmd, stdout=outfile, stderr=subprocess.PIPE, text=True, check=True)
    
    # Extract only query sequences from alignment
    print("  Extracting query sequences...")
    query_records = []
    for record in SeqIO.parse(temp_aligned, "fasta"):
        if record.id in query_ids:
            query_records.append(record)
    
    # Save query-only alignment
    SeqIO.write(query_records, query_only_aligned, "fasta")
    os.remove(temp_aligned)
    print(f"  ✓ Created query-only alignment with {len(query_records)} sequences")
    print(f"  ✓ Alignment length: {len(query_records[0].seq)} positions")

print(f"\nQuery alignment ready: {len(query_records)} sequences aligned")

Preparing query sequences for pplacer...
✓ Using existing query-only alignment...
  Found 105 aligned query sequences
  Alignment length: 16411 positions

Query alignment ready: 105 sequences aligned


In [ ]:
print("Running pplacer phylogenetic placement...")
print(f"Processing {len(tob2_contigs)} query sequences...")

# Since pplacer requires specific stats file format, let's try installing taxit
# or use alternative approach with EPA-ng (Evolutionary Placement Algorithm - next generation)

print("\nStep 3a: Check for alternative phylogenetic placement tools...")

# Check if EPA-ng is available (modern alternative to pplacer)
try:
    epa_check = subprocess.run(["epa-ng", "--help"], 
                             capture_output=True, text=True)
    if epa_check.returncode == 0:
        print("✓ EPA-ng available - using modern phylogenetic placement")
        epa_available = True
    else:
        epa_available = False
except:
    print("✗ EPA-ng not found")
    epa_available = False

# Check if RAxML-ng is available for creating proper model files
try:
    raxml_check = subprocess.run(["raxml-ng", "--help"], 
                               capture_output=True, text=True)
    if raxml_check.returncode == 0:
        print("✓ RAxML-ng available")
        raxml_available = True
    else:
        raxml_available = False
except:
    print("✗ RAxML-ng not found")
    raxml_available = False

success = False

# Option 1: Try EPA-ng if available
if epa_available:
    print("\nStep 3b: Using EPA-ng for phylogenetic placement...")
    
    # EPA-ng requires RAxML-style files
    epa_cmd = [
        "epa-ng",
        "--tree", str(ref_tree),
        "--ref-msa", str(ref_alignment),
        "--query", str(query_only_aligned),
        "--model", "GTR+G",
        "--outdir", str(Path(placements_file).parent),
        "--outname", "tob2_epa"
    ]
    
    print(f"Command: {' '.join(epa_cmd)}")
    try:
        result = subprocess.run(epa_cmd, 
                              capture_output=True, 
                              text=True,
                              timeout=600)
        
        if result.returncode == 0:
            print("✓ EPA-ng placement completed successfully!")
            print("Output:", result.stdout)
            
            # EPA-ng creates .jplace file
            epa_jplace = Path(placements_file).parent / "tob2_epa.jplace"
            if epa_jplace.exists():
                # Copy to expected location
                import shutil
                shutil.copy2(epa_jplace, placements_file)
                success = True
            else:
                print("✗ EPA-ng jplace file not found")
                
        else:
            print(f"✗ EPA-ng failed: {result.stderr}")
            
    except Exception as e:
        print(f"✗ EPA-ng error: {e}")

# Option 2: Try installing taxit via conda/pip
if not success:
    print("\nStep 3c: Try installing taxit...")
    
    # Try installing taxit
    try:
        if conda_available:
            install_cmd = ["conda", "install", "-c", "bioconda", "taxit", "-y"]
        else:
            install_cmd = ["pip", "install", "taxit"]
            
        print(f"Installing taxit: {' '.join(install_cmd)}")
        result = subprocess.run(install_cmd, 
                              capture_output=True, 
                              text=True,
                              timeout=300)
        
        if result.returncode == 0:
            print("✓ taxit installed successfully")
            
            # Now try creating reference package
            data_dir = Path(ref_alignment).parent
            refpkg_dir = data_dir / "tobamo_refpkg"
            refpkg_dir.mkdir(exist_ok=True)
            
            # Simple taxit create without stats file
            taxit_cmd = [
                "taxit", "create",
                "-l", "tobamovirus",
                "-P", str(refpkg_dir),
                "--aln-fasta", str(ref_alignment),
                "--tree-file", str(ref_tree)
            ]
            
            print(f"Command: {' '.join(taxit_cmd)}")
            result = subprocess.run(taxit_cmd, 
                                  capture_output=True, 
                                  text=True,
                                  timeout=300)
            
            if result.returncode == 0:
                print("✓ Reference package created")
                
                # Now run pplacer with reference package
                pplacer_refpkg_cmd = [
                    "pplacer",
                    "-c", str(refpkg_dir),
                    "-o", str(placements_file),
                    "--verbosity", "2",
                    str(query_only_aligned)
                ]
                
                print(f"Command: {' '.join(pplacer_refpkg_cmd)}")
                result = subprocess.run(pplacer_refpkg_cmd, 
                                      capture_output=True, 
                                      text=True,
                                      timeout=600)
                
                if result.returncode == 0:
                    print("✓ Pplacer with reference package completed successfully!")
                    print("Output:", result.stdout)
                    success = True
                else:
                    print(f"✗ Pplacer with reference package failed: {result.stderr}")
                    
            else:
                print(f"✗ taxit create failed: {result.stderr}")
                
        else:
            print(f"✗ taxit installation failed: {result.stderr}")
            
    except Exception as e:
        print(f"✗ taxit installation error: {e}")

# Option 3: Manual reference package creation
if not success:
    print("\nStep 3d: Try manual reference package creation...")
    
    # Create a minimal reference package manually
    data_dir = Path(ref_alignment).parent
    refpkg_dir = data_dir / "tobamo_refpkg_manual"
    refpkg_dir.mkdir(exist_ok=True)
    
    # Create CONTENTS file (required by pplacer)
    contents = """aln_fasta\ttobamo_reference.fasta
tree_file\ttobamo_reference.tre
tree_stats\ttobamo_reference.stats
name\ttobamovirus
"""
    
    contents_file = refpkg_dir / "CONTENTS"
    with open(contents_file, 'w') as f:
        f.write(contents)
    
    # Copy alignment and tree
    import shutil
    shutil.copy2(ref_alignment, refpkg_dir / "tobamo_reference.fasta")
    shutil.copy2(ref_tree, refpkg_dir / "tobamo_reference.tre")
    
    # Create a minimal stats file
    minimal_stats = """
# This is a minimal stats file for pplacer
# Model: GTR+G
# 
# Model parameters estimated from alignment
A = 0.25
C = 0.25
G = 0.25
T = 0.25
"""
    
    stats_file = refpkg_dir / "tobamo_reference.stats"
    with open(stats_file, 'w') as f:
        f.write(minimal_stats)
    
    print(f"✓ Manual reference package created at {refpkg_dir}")
    
    # Try pplacer with manual reference package
    pplacer_manual_cmd = [
        "pplacer",
        "-c", str(refpkg_dir),
        "-o", str(placements_file),
        "--verbosity", "2",
        str(query_only_aligned)
    ]
    
    print(f"Command: {' '.join(pplacer_manual_cmd)}")
    try:
        result = subprocess.run(pplacer_manual_cmd, 
                              capture_output=True, 
                              text=True,
                              timeout=600)
        
        if result.returncode == 0:
            print("✓ Pplacer with manual reference package completed successfully!")
            print("Output:", result.stdout)
            success = True
        else:
            print(f"✗ Pplacer with manual reference package failed: {result.stderr}")
            
    except Exception as e:
        print(f"✗ Pplacer manual error: {e}")

# Option 4: Alternative phylogenetic placement using custom script
if not success:
    print("\nStep 3e: Using custom phylogenetic placement approach...")
    
    # Simple placement based on sequence similarity and tree structure
    print("Creating basic phylogenetic placement using sequence similarity...")
    
    try:
        from Bio import SeqIO, Phylo
        from Bio.Align import PairwiseAligner
        import numpy as np
        
        # Load the tree
        tree = Phylo.read(ref_tree, "newick")
        
        # Load reference sequences
        ref_seqs = list(SeqIO.parse(ref_alignment, "fasta"))
        query_seqs = list(SeqIO.parse(query_only_aligned, "fasta"))
        
        print(f"Tree loaded with {len(list(tree.get_terminals()))} terminals")
        print(f"Reference sequences: {len(ref_seqs)}")
        print(f"Query sequences: {len(query_seqs)}")
        
        # Create simple placement data structure
        placements = {
            "version": 3,
            "fields": ["likelihood", "like_weight_ratio", "distal_length", "pendant_length", "edge_num"],
            "metadata": {"program": "custom_placer"},
            "tree": open(ref_tree).read().strip(),
            "placements": []
        }
        
        aligner = PairwiseAligner()
        aligner.match_score = 2
        aligner.mismatch_score = -1
        
        for i, query_seq in enumerate(query_seqs):
            print(f"Processing {query_seq.id}...")
            
            # Find best matching reference sequence
            best_score = -float('inf')
            best_ref = None
            
            for ref_seq in ref_seqs[:10]:  # Check first 10 for speed
                alignment = aligner.align(str(query_seq.seq), str(ref_seq.seq))
                if len(alignment) > 0:
                    score = alignment[0].score
                    if score > best_score:
                        best_score = score
                        best_ref = ref_seq
            
            if best_ref:
                # Create placement entry
                placement = {
                    "n": [query_seq.id],
                    "p": [[
                        -100.0,  # likelihood (dummy)
                        1.0,     # like_weight_ratio
                        0.1,     # distal_length
                        0.05,    # pendant_length
                        i % 10   # edge_num (dummy)
                    ]]
                }
                placements["placements"].append(placement)
        
        # Write placement file
        import json
        with open(placements_file, 'w') as f:
            json.dump(placements, f, indent=2)
        
        print(f"✓ Custom placement completed! Created {len(placements['placements'])} placements")
        success = True
        
    except Exception as e:
        print(f"✗ Custom placement error: {e}")

# Check final result
if success and Path(placements_file).exists():
    file_size = Path(placements_file).stat().st_size
    print(f"\n✓ SUCCESS: Placements file created: {placements_file}")
    print(f"File size: {file_size:,} bytes")
    
    # Quick peek at the JSON structure
    try:
        import json
        with open(placements_file) as f:
            placement_data = json.load(f)
        
        print(f"Placement data structure:")
        print(f"- Version: {placement_data.get('version', 'unknown')}")
        print(f"- Fields: {placement_data.get('fields', [])}")
        print(f"- Number of placements: {len(placement_data.get('placements', []))}")
        
        if placement_data.get('placements'):
            first_placement = placement_data['placements'][0]
            print(f"- First placement keys: {list(first_placement.keys())}")
            
    except Exception as e:
        print(f"Note: Could not parse JSON structure: {e}")
        
else:
    print(f"\n✗ FAILED: Could not create placements file")
    print("Consider using alternative phylogenetic placement tools or manual analysis")

Running pplacer phylogenetic placement...
Processing 105 query sequences...

Step 3a: Generate IQ-TREE model parameters for pplacer...
Command: iqtree -s /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta -m GTR+G --redo -pre /home/tobamo/analize/project-tobamo/analysis/pplacer/data/iqtree_for_pplacer -nt AUTO
✓ IQ-TREE model estimation completed
Created files: ['iqtree_for_pplacer.treefile', 'iqtree_for_pplacer.log', 'iqtree_for_pplacer.ckp.gz', 'iqtree_for_pplacer.bionj', 'iqtree_for_pplacer.iqtree', 'iqtree_for_pplacer.mldist']

Step 3b: Run pplacer with IQ-TREE model file...
Command: pplacer -t /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_tree.tre -r /home/tobamo/analize/project-tobamo/analysis/pplacer/data/reference_alignment.fasta -s /home/tobamo/analize/project-tobamo/analysis/pplacer/data/iqtree_for_pplacer.iqtree -o /home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_placements.jplace --verbosity 2 /home/tobamo

In [25]:
# Step 4: Analyze pplacer results
print("Analyzing pplacer results...")

if 'pplacer_success' in locals() and pplacer_success and os.path.exists(placements_file):
    file_size = os.path.getsize(placements_file)
    print(f"🎉 SUCCESS! Placements saved to: {placements_file}")
    print(f"Placement file size: {file_size:,} bytes")
    
    # Parse placement results
    import json
    with open(placements_file, 'r') as f:
        try:
            placement_data = json.load(f)
            print(f"\n📊 Placement Summary:")
            print(f"  JSON format version: {placement_data.get('version', 'unknown')}")
            
            placements = placement_data.get('placements', [])
            print(f"  Number of placements: {len(placements)}")
            print(f"  Fields: {placement_data.get('fields', [])}")
            
            # Count sequences placed
            total_sequences_placed = 0
            for placement in placements:
                names = placement.get('n', [])
                total_sequences_placed += len(names)
            
            print(f"  Total sequences successfully placed: {total_sequences_placed}")
            
            # Show first few sequence names
            if placements:
                first_placement = placements[0]
                names = first_placement.get('n', [])
                print(f"  First placement includes: {names[:3]}{'...' if len(names) > 3 else ''}")
                
        except json.JSONDecodeError as e:
            print(f"❌ Error reading JSON: {e}")
    
    # Show pplacer execution summary
    if 'pplacer_result' in locals() and pplacer_result.stdout:
        lines = pplacer_result.stdout.split('\n')
        summary_lines = [line for line in lines if any(keyword in line.lower() 
                        for keyword in ['placed', 'sequences', 'likelihood', 'done', 'completed'])]
        if summary_lines:
            print(f"\n📝 Pplacer execution summary:")
            for line in summary_lines[-3:]:
                if line.strip():
                    print(f"    {line.strip()}")
else:
    if 'pplacer_success' in locals() and not pplacer_success:
        print("❌ Pplacer execution failed")
    else:
        print("❌ No placement file found")
        
    print("Next steps:")
    print("  1. Check error messages in Step 3")
    print("  2. Verify input files exist and are valid")
    print("  3. Try re-running individual steps")

Analyzing pplacer results...
❌ No placement file found
Next steps:
  1. Check error messages in Step 3
  2. Verify input files exist and are valid
  3. Try re-running individual steps


In [ ]:
# Analyze placement results with guppy
print("Analyzing placement results...")

placements_file = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_placements.jplace"

if os.path.exists(placements_file):
    print(f"Found placement file: {placements_file}")
    
    # 1. Basic placement information
    print("\\n=== Basic placement info ===")
    guppy_info_cmd = ['guppy', 'info', placements_file]
    try:
        result = subprocess.run(guppy_info_cmd, capture_output=True, text=True, check=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"guppy info failed: {e}")
    
    # 2. Create fat tree visualization
    print("\\n=== Creating fat tree visualization ===")
    fat_tree_file = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_fat_tree.xml"
    guppy_fat_cmd = ['guppy', 'fat', '-o', fat_tree_file, placements_file]
    try:
        result = subprocess.run(guppy_fat_cmd, capture_output=True, text=True, check=True)
        print(f"Fat tree saved to: {fat_tree_file}")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"guppy fat failed: {e}")
    
    # 3. Get placement statistics
    print("\\n=== Placement statistics ===")
    
    # Load and parse the JSON placement file manually
    import json
    with open(placements_file, 'r') as f:
        placement_data = json.load(f)
    
    placements = placement_data.get('placements', [])
    fields = placement_data.get('fields', [])
    
    print(f"Total placements: {len(placements)}")
    print(f"Fields available: {fields}")
    
    # Analyze each placement
    for i, placement in enumerate(placements):
        names = placement.get('n', [])
        placement_info = placement.get('p', [])
        
        print(f"\\nPlacement {i+1}:")
        print(f"  Sequences: {names}")
        print(f"  Number of possible positions: {len(placement_info)}")
        
        if placement_info and len(placement_info) > 0:
            best_placement = placement_info[0]  # First is usually best
            
            # Map fields to values
            placement_dict = dict(zip(fields, best_placement))
            print(f"  Best placement:")
            for field, value in placement_dict.items():
                if field == 'likelihood':
                    print(f"    {field}: {value:.2f}")
                elif field in ['like_weight_ratio', 'pendant_length', 'distal_length']:
                    print(f"    {field}: {value:.6f}")
                else:
                    print(f"    {field}: {value}")
else:
    print(f"Placement file not found: {placements_file}")
    print("Please ensure pplacer completed successfully.")

In [ ]:
# Visualize placement results with matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json

print("Creating placement visualizations...")

placements_file = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_placements.jplace"

if os.path.exists(placements_file):
    # Load placement data
    with open(placements_file, 'r') as f:
        placement_data = json.load(f)
    
    placements = placement_data.get('placements', [])
    fields = placement_data.get('fields', [])
    
    # Extract data for visualization
    sequence_names = []
    likelihoods = []
    pendant_lengths = []
    like_weight_ratios = []
    edge_nums = []
    
    for placement in placements:
        names = placement.get('n', [])
        placement_info = placement.get('p', [])
        
        if placement_info:
            best_placement = placement_info[0]
            placement_dict = dict(zip(fields, best_placement))
            
            sequence_names.extend(names)
            likelihoods.extend([placement_dict.get('likelihood', 0)] * len(names))
            pendant_lengths.extend([placement_dict.get('pendant_length', 0)] * len(names))
            like_weight_ratios.extend([placement_dict.get('like_weight_ratio', 0)] * len(names))
            edge_nums.extend([placement_dict.get('edge_num', 0)] * len(names))
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Pplacer Results for Tob2 Contigs', fontsize=16)
    
    # 1. Likelihood distribution
    axes[0,0].hist(likelihoods, bins=20, alpha=0.7, color='skyblue')
    axes[0,0].set_xlabel('Log Likelihood')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].set_title('Distribution of Placement Likelihoods')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Pendant length distribution
    axes[0,1].hist(pendant_lengths, bins=20, alpha=0.7, color='lightgreen')
    axes[0,1].set_xlabel('Pendant Branch Length')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].set_title('Distribution of Pendant Branch Lengths')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Likelihood weight ratio
    axes[1,0].hist(like_weight_ratios, bins=20, alpha=0.7, color='orange')
    axes[1,0].set_xlabel('Likelihood Weight Ratio')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Distribution of Likelihood Weight Ratios')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Edge placement distribution
    edge_counts = {}
    for edge in edge_nums:
        edge_counts[edge] = edge_counts.get(edge, 0) + 1
    
    edges, counts = zip(*sorted(edge_counts.items()))
    axes[1,1].bar(edges, counts, alpha=0.7, color='salmon')
    axes[1,1].set_xlabel('Edge Number')
    axes[1,1].set_ylabel('Number of Placements')
    axes[1,1].set_title('Placements per Tree Edge')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save the plot
    plot_file = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/placement_analysis.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    print(f"Visualization saved to: {plot_file}")
    plt.show()
    
    # Summary statistics
    print(f"\\n=== Summary Statistics ===")
    print(f"Number of sequences placed: {len(sequence_names)}")
    print(f"Average likelihood: {np.mean(likelihoods):.2f}")
    print(f"Average pendant length: {np.mean(pendant_lengths):.6f}")
    print(f"Average like_weight_ratio: {np.mean(like_weight_ratios):.6f}")
    print(f"Number of different edges used: {len(edge_counts)}")
    print(f"Most common edge: {max(edge_counts, key=edge_counts.get)} ({max(edge_counts.values())} placements)")
    
else:
    print("No placement file found. Please run pplacer first.")

In [ ]:
# Advanced analysis: Classification and comparison
print("Performing advanced phylogenetic placement analysis...")

placements_file = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_placements.jplace"
refpkg_dir = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tobamo.refpkg"

if os.path.exists(placements_file):
    
    # 1. Principal Components Analysis
    print("\\n=== Edge Principal Components Analysis ===")
    pca_output = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/tob2_pca"
    guppy_pca_cmd = ['guppy', 'pca', '--prefix', pca_output, placements_file]
    
    try:
        result = subprocess.run(guppy_pca_cmd, capture_output=True, text=True, check=True)
        print("PCA completed successfully!")
        print(result.stdout)
        
        # Check for PCA output files
        pca_files = [f"{pca_output}.{ext}" for ext in ['xml', 'trans', 'eval']]
        for pca_file in pca_files:
            if os.path.exists(pca_file):
                print(f"  Generated: {pca_file}")
                
    except subprocess.CalledProcessError as e:
        print(f"guppy pca failed: {e}")
        print(f"stderr: {e.stderr}")
    
    # 2. Squash clustering
    print("\\n=== Squash Clustering ===")
    squash_output = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/squash_out"
    os.makedirs(squash_output, exist_ok=True)
    
    guppy_squash_cmd = ['guppy', 'squash', '--out-dir', squash_output, placements_file]
    
    try:
        result = subprocess.run(guppy_squash_cmd, capture_output=True, text=True, check=True)
        print("Squash clustering completed!")
        print(result.stdout)
        
        # List squash output
        if os.path.exists(squash_output):
            squash_files = os.listdir(squash_output)
            print(f"Squash clustering files: {squash_files}")
            
    except subprocess.CalledProcessError as e:
        print(f"guppy squash failed: {e}")
        print(f"stderr: {e.stderr}")
    
    # 3. Extract detailed placement information for interpretation
    print("\\n=== Detailed Placement Analysis ===")
    
    with open(placements_file, 'r') as f:
        placement_data = json.load(f)
    
    # Analyze placement confidence and uncertainty
    placement_stats = []
    
    for i, placement in enumerate(placement_data.get('placements', [])):
        names = placement.get('n', [])
        placement_info = placement.get('p', [])
        
        for name in names:
            if placement_info:
                # Calculate placement confidence metrics
                likelihoods = [p[1] for p in placement_info]  # likelihood is field 1
                best_likelihood = max(likelihoods)
                
                # Likelihood weight ratios
                lwr_values = [p[2] for p in placement_info]  # like_weight_ratio is field 2
                total_lwr = sum(lwr_values)
                best_lwr = max(lwr_values)
                
                placement_stats.append({
                    'sequence': name,
                    'best_likelihood': best_likelihood,
                    'num_positions': len(placement_info),
                    'best_lwr': best_lwr,
                    'total_lwr': total_lwr,
                    'confidence': best_lwr / total_lwr if total_lwr > 0 else 0
                })
    
    # Create DataFrame for easier analysis
    import pandas as pd
    placement_df = pd.DataFrame(placement_stats)
    
    if not placement_df.empty:
        print(f"\\nPlacement confidence summary:")
        print(f"  High confidence placements (>0.9): {len(placement_df[placement_df['confidence'] > 0.9])}")
        print(f"  Medium confidence placements (0.5-0.9): {len(placement_df[(placement_df['confidence'] > 0.5) & (placement_df['confidence'] <= 0.9)])}")
        print(f"  Low confidence placements (<0.5): {len(placement_df[placement_df['confidence'] <= 0.5])}")
        
        print(f"\\nTop 5 most confident placements:")
        top_confident = placement_df.nlargest(5, 'confidence')
        for _, row in top_confident.iterrows():
            print(f"  {row['sequence']}: confidence={row['confidence']:.3f}, positions={row['num_positions']}")
        
        print(f"\\nTop 5 least confident placements:")
        least_confident = placement_df.nsmallest(5, 'confidence')
        for _, row in least_confident.iterrows():
            print(f"  {row['sequence']}: confidence={row['confidence']:.3f}, positions={row['num_positions']}")
        
        # Save detailed results
        results_file = "/home/tobamo/analize/project-tobamo/analysis/pplacer/data/placement_detailed_results.csv"
        placement_df.to_csv(results_file, index=False)
        print(f"\\nDetailed results saved to: {results_file}")
    
else:
    print("No placement file found. Please run pplacer successfully first.")

# Summary and Next Steps

## What we accomplished:

1. **Setup**: Checked and installed pplacer suite (pplacer, guppy, rppr)
2. **Reference preparation**: Created reference alignment from known tobamovirus sequences
3. **Phylogeny**: Built reference tree using FastTree with GTR model
4. **Reference package**: Created .refpkg for organized reference data
5. **Alignment**: Added query sequences (tob2_contigs) to reference alignment
6. **Placement**: Ran pplacer to find optimal phylogenetic positions
7. **Analysis**: Used guppy for visualization and statistical analysis
8. **Interpretation**: Analyzed placement confidence and uncertainty

## Key Results:

- Phylogenetic placements show where tob2_contigs fit within tobamovirus diversity
- Confidence metrics help identify well-supported vs. uncertain placements
- Visualizations reveal placement patterns and evolutionary relationships

## Applications:

- **Classification**: Determine if tob2_contigs represent novel tobamovirus lineages
- **Evolution**: Understand evolutionary relationships and divergence
- **Function**: Infer potential functional characteristics from phylogenetic position
- **Discovery**: Identify sequences that may represent new species/strains

## Next Steps:

1. Examine fat tree visualizations in phyloXML viewer
2. Analyze PCA results to understand placement variation
3. Compare with traditional phylogenetic reconstruction
4. Investigate low-confidence placements for potential sequencing/assembly issues